In [1]:
%matplotlib inline

from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import menpo.io as mio
from menpo.math import log_gabor
from menpo.feature import fast_dsift
from menpo.image import Image
from menpo.visualize import print_dynamic, progress_bar_str, visualize_images
from menpofit.visualize import visualize_fitting_results
from alaborticcv2015.alignment import (
    LKFitter, 
    FilteredSSD, FilteredFourierSSD, ECC, GradientImages, GradientCorrelation)
from alaborticcv2015.alignment.result import SerializableResult

# Experiment 3: AR Database, Previous Methods

In [13]:
db = mio.import_pickle('/Users/joan/Desktop/alaborticcv2015/data/ar.pkl.gz')

In [14]:
folder_path = '/Users/joan/Desktop/alaborticcv2015/results/ar/'

diagonal = 100
scales = (1,)
max_iters = 100

n_rep = 20
noise_std = range(2, 20, 2)

## SSD 

In [15]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))
for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=FilteredSSD)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + 'ar_ssd.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  img: 2,  mean: 0.0155 - 0.1086  median: 0.0157 - 0.1106  std: 0.0043 - 0.0561 
- noise: 2,  sub: 1,  img: 2,  mean: 0.0155 - 0.0314  median: 0.0157 - 0.0251  std: 0.0043 - 0.0097 
- noise: 2,  sub: 2,  img: 2,  mean: 0.0155 - 0.0303  median: 0.0157 - 0.0247  std: 0.0043 - 0.0138 
- noise: 2,  sub: 3,  img: 2,  mean: 0.0154 - 0.3793  median: 0.0156 - 0.2740  std: 0.0043 - 0.3504 
- noise: 2,  sub: 4,  img: 2,  mean: 0.0155 - 0.2328  median: 0.0157 - 0.2669  std: 0.0043 - 0.1274 
- noise: 2,  sub: 5,  img: 2,  mean: 0.0155 - 0.2235  median: 0.0157 - 0.1853  std: 0.0043 - 0.0982 
- noise: 2,  sub: 6,  img: 2,  mean: 0.0155 - 0.2119  median: 0.0157 - 0.2311  std: 0.0043 - 0.0644 
- noise: 2,  sub: 7,  img: 2,  mean: 0.0155 - 0.2721  median: 0.0157 - 0.3118  std: 0.0043 - 0.1084 
- noise: 2,  sub: 8,  img: 2,  mean: 0.0154 - 0.2391  median: 0.0157 - 0.2836  std: 0.0043 - 0.1656 
- noise: 2,  sub: 9,  img: 2,  mean: 0.0154 - 0.2796  median: 0.0156 - 0.1295  std: 0.0043 

## LogGabor 

In [11]:
def kernel_func(shape):
    k = np.fft.ifftshift(log_gabor(np.ones(shape[-2:]))[2]) 
    return k

errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))
for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=FilteredFourierSSD,
                          kernel_func=kernel_func)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + 'ar_loggabor.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  img: 2,  mean: 0.0118 - 0.2161  median: 0.0119 - 0.1693  std: 0.0032 - 0.0724 
- noise: 2,  sub: 1,  img: 2,  mean: 0.0115 - 0.1296  median: 0.0116 - 0.0396  std: 0.0032 - 0.1366 
- noise: 2,  sub: 2,  img: 2,  mean: 0.0094 - 0.0366  median: 0.0095 - 0.0413  std: 0.0026 - 0.0158 
- noise: 2,  sub: 3,  img: 2,  mean: 0.0106 - 0.0873  median: 0.0108 - 0.0904  std: 0.0029 - 0.0572 
- noise: 2,  sub: 4,  img: 2,  mean: 0.0115 - 0.2018  median: 0.0116 - 0.1994  std: 0.0032 - 0.0090 
- noise: 2,  sub: 5,  img: 2,  mean: 0.0106 - 0.3747  median: 0.0108 - 0.3704  std: 0.0029 - 0.0271 
- noise: 2,  sub: 6,  img: 2,  mean: 0.0101 - 1.1722  median: 0.0103 - 1.1579  std: 0.0028 - 0.8250 
- noise: 2,  sub: 7,  img: 2,  mean: 0.0102 - 0.1925  median: 0.0104 - 0.2559  std: 0.0028 - 0.0941 
- noise: 2,  sub: 8,  img: 2,  mean: 0.0107 - 0.2157  median: 0.0109 - 0.2831  std: 0.0030 - 0.1306 
- noise: 2,  sub: 9,  img: 2,  mean: 0.0109 - 0.4238  median: 0.0110 - 0.4401  std: 0.0030 

## ECC 

In [8]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=ECC)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + 'ar_ecc.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  img: 2,  mean: 0.0118 - 0.2341  median: 0.0119 - 0.2981  std: 0.0032 - 0.0997 
- noise: 2,  sub: 1,  img: 2,  mean: 0.0115 - 0.2089  median: 0.0116 - 0.2112  std: 0.0032 - 0.1374 
- noise: 2,  sub: 2,  img: 2,  mean: 0.0094 - 0.0395  median: 0.0095 - 0.0389  std: 0.0026 - 0.0141 
- noise: 2,  sub: 3,  img: 2,  mean: 0.0106 - 0.1220  median: 0.0108 - 0.0269  std: 0.0029 - 0.1409 
- noise: 2,  sub: 4,  img: 2,  mean: 0.0115 - 0.2146  median: 0.0116 - 0.1964  std: 0.0032 - 0.0294 
- noise: 2,  sub: 5,  img: 2,  mean: 0.0106 - 0.1866  median: 0.0108 - 0.0606  std: 0.0029 - 0.1927 


ValueError: Image has 0 variance - can't be normalized

## Gradient Images 

In [7]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=GradientImages)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + 'ar_gi.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  img: 2,  mean: 0.0118 - 0.0274  median: 0.0119 - 0.0277  std: 0.0032 - 0.0049 
- noise: 2,  sub: 1,  img: 2,  mean: 0.0115 - 0.0102  median: 0.0116 - 0.0104  std: 0.0032 - 0.0020 
- noise: 2,  sub: 2,  img: 2,  mean: 0.0094 - 0.0090  median: 0.0095 - 0.0100  std: 0.0026 - 0.0035 
- noise: 2,  sub: 3,  img: 2,  mean: 0.0106 - 0.0070  median: 0.0108 - 0.0059  std: 0.0029 - 0.0032 
- noise: 2,  sub: 4,  img: 2,  mean: 0.0115 - 0.0106  median: 0.0116 - 0.0095  std: 0.0032 - 0.0034 
- noise: 2,  sub: 5,  img: 2,  mean: 0.0106 - 0.0081  median: 0.0108 - 0.0094  std: 0.0029 - 0.0022 
- noise: 2,  sub: 6,  img: 2,  mean: 0.0101 - 0.0284  median: 0.0103 - 0.0332  std: 0.0028 - 0.0108 
- noise: 2,  sub: 7,  img: 2,  mean: 0.0102 - 0.0174  median: 0.0104 - 0.0165  std: 0.0028 - 0.0023 
- noise: 2,  sub: 8,  img: 2,  mean: 0.0107 - 0.0198  median: 0.0109 - 0.0229  std: 0.0030 - 0.0061 
- noise: 2,  sub: 9,  img: 2,  mean: 0.0109 - 0.0130  median: 0.0110 - 0.0133  std: 0.0030 

## Gradient Correlation

In [6]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=GradientCorrelation)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string2 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + 'ar_gc.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  img: 2,  mean: 0.0118 - 0.0364  median: 0.0119 - 0.0358  std: 0.0032 - 0.0081 
- noise: 2,  sub: 1,  img: 2,  mean: 0.0115 - 0.0158  median: 0.0116 - 0.0156  std: 0.0032 - 0.0026 
- noise: 2,  sub: 2,  img: 2,  mean: 0.0094 - 0.0099  median: 0.0095 - 0.0073  std: 0.0026 - 0.0068 
- noise: 2,  sub: 3,  img: 2,  mean: 0.0106 - 0.0070  median: 0.0108 - 0.0065  std: 0.0029 - 0.0022 
- noise: 2,  sub: 4,  img: 2,  mean: 0.0115 - 0.0123  median: 0.0116 - 0.0124  std: 0.0032 - 0.0029 
- noise: 2,  sub: 5,  img: 2,  mean: 0.0106 - 0.0113  median: 0.0108 - 0.0136  std: 0.0029 - 0.0045 
- noise: 2,  sub: 6,  img: 2,  mean: 0.0101 - 0.0223  median: 0.0103 - 0.0257  std: 0.0028 - 0.0121 
- noise: 2,  sub: 7,  img: 2,  mean: 0.0102 - 0.0118  median: 0.0104 - 0.0118  std: 0.0028 - 0.0014 
- noise: 2,  sub: 8,  img: 2,  mean: 0.0107 - 0.0137  median: 0.0109 - 0.0142  std: 0.0030 - 0.0040 
- noise: 2,  sub: 9,  img: 2,  mean: 0.0109 - 0.0162  median: 0.0110 - 0.0150  std: 0.0030 

## dSift 

In [4]:
errors = np.empty((len(noise_std), len(db), len(db[0]['images']), n_rep, max_iters+1))

for l, n in enumerate(noise_std):
    string = '- noise: {}, '.format(n)
    
    for sub in range(len(db)):
        string1 = string + ' sub: {}, '.format(sub)

        subject = db[sub]
        template = subject['template']
        test_images = subject['images']
        
        fitter = LKFitter(template,
                          group='bounding_box',
                          features=fast_dsift,
                          diagonal=diagonal,
                          scales=scales,
                          residual_cls=FilteredSSD)

        np.random.seed(0)
        for j, i in enumerate(test_images):
            string2 = string1 + ' img: {}, '.format(j)
            
            for k in range(n_rep):
                gt_s = i.landmarks['bounding_box'].lms
                s = fitter.perturb_shape(gt_s, noise_std=n)
                fr = fitter.fit(i, s, gt_shape=gt_s, max_iters=max_iters)
                fr.downscale = 0.5
                errors[l, sub, j, k] = fr.errors()
                
            
            ini_errors = errors[l, sub, j, :, 0]
            ini_mean = np.mean(ini_errors)
            ini_median = np.median(ini_errors)
            ini_std = np.std(ini_errors)
            
            final_errors = errors[l, sub, j, :, -1]
            final_mean = np.mean(final_errors)
            final_median = np.median(final_errors)
            final_std = np.std(final_errors)
            
            print_dynamic(string2 + 
                          ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                          ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                          ' std: {0:.4f} - {1:.4f} '.format(ini_std, final_std)) 

        ini_errors = errors[l, sub, :, :, 0]
        ini_mean = np.mean(ini_errors)
        ini_median = np.median(ini_errors)
        ini_std = np.std(ini_errors)

        final_errors = errors[l, sub, :, :, -1]
        final_mean = np.mean(final_errors)
        final_median = np.median(final_errors)
        final_std = np.std(final_errors)

        print_dynamic(string1 + 
                      ' mean: {0:.4f} - {1:.4f} '.format(ini_mean, final_mean) + 
                      ' median: {0:.4f} - {1:.4f} '.format(ini_median, final_median) + 
                      ' std: {0:.4f} - {1:.4f} \n'.format(ini_std, final_std))
        
mio.export_pickle(errors, folder_path + 'ar_dsift.pkl.gz', overwrite=True)

- noise: 2,  sub: 0,  mean: 0.0118 - 0.0196  median: 0.0119 - 0.0196  std: 0.0032 - 0.0059 
- noise: 2,  sub: 1,  mean: 0.0115 - 0.0223  median: 0.0116 - 0.0173  std: 0.0032 - 0.0099 
- noise: 2,  sub: 2,  mean: 0.0094 - 0.0159  median: 0.0095 - 0.0147  std: 0.0026 - 0.0062 
- noise: 2,  sub: 3,  mean: 0.0106 - 0.0119  median: 0.0108 - 0.0099  std: 0.0029 - 0.0035 
- noise: 2,  sub: 4,  mean: 0.0115 - 0.0389  median: 0.0116 - 0.0502  std: 0.0032 - 0.0172 
- noise: 2,  sub: 5,  mean: 0.0106 - 0.0116  median: 0.0108 - 0.0116  std: 0.0029 - 0.0014 
- noise: 2,  sub: 6,  mean: 0.0101 - 0.0264  median: 0.0103 - 0.0257  std: 0.0028 - 0.0022 
- noise: 2,  sub: 7,  mean: 0.0102 - 0.0273  median: 0.0104 - 0.0244  std: 0.0028 - 0.0062 
- noise: 2,  sub: 8,  mean: 0.0107 - 0.0151  median: 0.0109 - 0.0150  std: 0.0030 - 0.0040 
- noise: 2,  sub: 9,  mean: 0.0109 - 0.0185  median: 0.0110 - 0.0201  std: 0.0030 - 0.0036 
- noise: 2,  sub: 10,  mean: 0.0105 - 0.0127  median: 0.0106 - 0.0090  std: 0.00

IndexError: list index out of range

In [ ]:
def mean_error_per_noise_iter(errors):
    return np.mean(errors, axis=(0, 1, 2))

def plot_mean_error_per_noise_iter(errors):
    errors = mean_error_per_noise_iter(errors)
    
    plt.figure()
    plt.title('Normalized error per iteration')
    plt.ylabel('Normalized error')
    plt.xlabel('Iteration')
    plt.plot(range(errors.shape[0]), errors)

def convergence_per_noise(errors, threshold=0.03):
    final_errors = errors[..., -1]
    convergence_errors = np.empty(final_errors.shape[0])
    for j, errors in enumerate(final_errors):
        n_errors = np.prod(errors.shape)
        convergence_errors[j] = np.count_nonzero([errors <= threshold]) / n_errors
    return convergence_errors

def plot_convergence_per_noise(errors, threshold=0.03):
    errors = convergence_per_noise(errors, threshold=threshold)
    
    plt.figure()
    plt.ylabel('% converged')
    plt.xlabel('Noise sigma')
    plt.plot(range(errors.shape[0]), errors)